# Frequentist Model Selection with AIC/BIC

This notebook demonstrates systematic model comparison using **frequentist information criteria** (AIC, BIC) and the `ModelComparisonPipeline` API. This complements the Bayesian approach (WAIC/LOO) shown in `bayesian/04-model-comparison.ipynb`.

**Learning Objectives:**
- Use `ModelComparisonPipeline` for automated model comparison
- Understand AIC (Akaike Information Criterion) and BIC (Bayesian Information Criterion)
- Calculate AIC weights and evidence ratios for model selection
- Interpret complexity vs performance trade-offs
- Compare frequentist (AIC/BIC) vs Bayesian (WAIC/LOO) model selection

**Prerequisites:** Basic understanding of model fitting (complete basic/ notebooks first)

**Estimated Time:** 40-45 minutes

**Companion Notebook:** Use `bayesian/04-model-comparison.ipynb` for Bayesian model selection with WAIC/LOO

**Key Concepts:**
- **AIC** (Akaike Information Criterion): Penalizes complexity based on number of parameters
- **BIC** (Bayesian Information Criterion): Stronger complexity penalty than AIC
- **ΔAIC < 2**: Models essentially equivalent
- **2 < ΔAIC < 4**: Moderate evidence for best model
- **4 < ΔAIC < 7**: Strong evidence for best model
- **ΔAIC > 10**: Very strong evidence for best model

In [ ]:
# Google Colab Setup - Run this cell first!
# Skip if running locally with rheojax already installed

import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Install rheojax and dependencies
    !pip install -q rheojax
    
    # Colab uses float32 by default - we need float64 for numerical stability
    # This MUST be set before importing JAX
    import os
    os.environ['JAX_ENABLE_X64'] = 'true'
    
    print("✓ RheoJAX installed successfully!")
    print("✓ Float64 precision enabled")

## Setup and Imports

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

import warnings

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

from rheojax.core.data import RheoData
from rheojax.pipeline import ModelComparisonPipeline

np.random.seed(42)

# Suppress matplotlib backend warning in VS Code
warnings.filterwarnings('ignore', message='.*non-interactive.*')


## Generate Synthetic Polymer Relaxation Data

We'll create time-domain relaxation data from a **FractionalZenerSolidSolid** model with known parameters. Relaxation mode is more stable than oscillation mode for optimization, ensuring reliable model comparison.

**Why use relaxation mode?**
- More stable optimization landscape (convex near global minimum)
- Fractional models converge reliably without local minima issues
- Demonstrates AIC/BIC model selection principles effectively

In [ ]:
# Generate time vector
time = np.logspace(-2, 2, 50)  # 0.01 to 100 seconds

# Import the true underlying model
from rheojax.models import FractionalZenerSolidSolid

# True parameters for Fractional Zener Solid-Solid model
Ge_true = 1e5      # Equilibrium modulus (Pa)
Gm_true = 5e5      # Maxwell arm modulus (Pa)
alpha_true = 0.6   # Fractional order
tau_alpha_true = 1.0  # Relaxation time (s^α)

# Create model and set true parameters
true_model = FractionalZenerSolidSolid()
true_model.set_params(Ge=Ge_true, Gm=Gm_true, alpha=alpha_true, tau_alpha=tau_alpha_true)

# Generate clean relaxation modulus using actual model equations
G_t_clean = true_model._predict_relaxation(
    time, Ge_true, Gm_true, alpha_true, tau_alpha_true
)

# Add realistic noise (5%)
noise_level = 0.05
G_t_noisy = G_t_clean * (1 + noise_level * np.random.randn(len(G_t_clean)))

# Create RheoData object
data = RheoData(
    x=time,
    y=G_t_noisy,
    x_units='s',
    y_units='Pa',
    domain='time',
)

print(f"Generated {len(time)} data points")
print(f"Time range: {time[0]:.2e} to {time[-1]:.2e} s")
print(f"Relaxation modulus range: {G_t_noisy.min():.2e} to {G_t_noisy.max():.2e} Pa")
print(f"\nTrue model: FractionalZenerSolidSolid")
print(f"  Ge        = {Ge_true:.2e} Pa")
print(f"  Gm        = {Gm_true:.2e} Pa")
print(f"  alpha     = {alpha_true:.2f}")
print(f"  tau_alpha = {tau_alpha_true:.2f} s^α")

## Visualize the Data

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Plot relaxation modulus
ax.loglog(time, G_t_noisy, 'o', color='steelblue', 
          label='Data (with noise)', markersize=6, alpha=0.7)
ax.loglog(time, G_t_clean, '--', color='red', 
          label='True model', linewidth=2)

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Relaxation Modulus G(t) (Pa)', fontsize=12)
ax.set_title('Relaxation Modulus Data', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## Define Candidate Models

We'll compare 5 models of increasing complexity:
1. **Maxwell**: Simplest viscoelastic model
2. **Zener**: Standard Linear Solid with equilibrium modulus
3. **Fractional Maxwell (Gel)**: Power-law relaxation
4. **Fractional Maxwell (Liquid)**: Fractional Maxwell without equilibrium modulus
5. **Fractional Zener (SS)**: Most complex, with fractional elements

In [ ]:
# Define candidate models
candidate_models = [
    'maxwell',
    'zener',
    'fractional_maxwell_gel',
    'fractional_maxwell_liquid',
    'fractional_zener_ss'
]

print(f"Comparing {len(candidate_models)} models:")
for i, model in enumerate(candidate_models, 1):
    print(f"  {i}. {model}")

## Run Model Comparison Pipeline

The `ModelComparisonPipeline` automatically fits all models and computes comparison metrics.

In [ ]:
# Create and run comparison pipeline
comparison_pipeline = ModelComparisonPipeline(candidate_models)
comparison_pipeline.run(data, test_mode="relaxation")

print("\nModel comparison complete!")
print(f"Fitted {len(comparison_pipeline.results)} models successfully")

## Display Comparison Table

We'll create a comprehensive comparison table with:
- **RMSE**: Root Mean Square Error (lower is better)
- **R²**: Coefficient of determination (higher is better, max 1.0)
- **AIC**: Akaike Information Criterion (lower is better)
- **BIC**: Bayesian Information Criterion (lower is better)

In [ ]:
# Extract metrics for each model
comparison_data = []

for model_name in candidate_models:
    if model_name in comparison_pipeline.results:
        result = comparison_pipeline.results[model_name]
        comparison_data.append({
            'Model': model_name,
            'Parameters': result['n_params'],
            'RMSE': result['rmse'],
            'R²': result['r_squared'],
            'AIC': result['aic'],
            'BIC': result['bic']
        })

# Create DataFrame
df_comparison = pd.DataFrame(comparison_data)

# Sort by AIC (lower is better)
df_comparison_sorted = df_comparison.sort_values('AIC').reset_index(drop=True)

# Display formatted table
print("\n" + "="*80)
print("MODEL COMPARISON RESULTS (sorted by AIC)")
print("="*80)
print()
print(df_comparison_sorted.to_string(index=False))
print("\n" + "="*80)
print("Lower AIC/BIC and RMSE are better; Higher R² is better (max 1.0)")
print("="*80)

## Identify Best Model

In [ ]:
# Get best model by different criteria
best_aic = comparison_pipeline.get_best_model(metric='aic')
best_bic = comparison_pipeline.get_best_model(metric='bic')
best_r2 = comparison_pipeline.get_best_model(metric='r_squared')

print("\n" + "="*60)
print("BEST MODEL BY DIFFERENT CRITERIA")
print("="*60)
print(f"\nBest by AIC:  {best_aic}")
print(f"Best by BIC:  {best_bic}")
print(f"Best by R²:   {best_r2}")

# Calculate AIC weights (relative likelihood)
aic_values = df_comparison_sorted['AIC'].values
delta_aic = aic_values - aic_values.min()
aic_weights = np.exp(-0.5 * delta_aic) / np.sum(np.exp(-0.5 * delta_aic))

print(f"\n{'Model':<30} {'AIC Weight':<15} {'Evidence Ratio'}")
print("-" * 60)
for i, (model, weight) in enumerate(zip(df_comparison_sorted['Model'], aic_weights)):
    evidence_ratio = aic_weights[0] / weight if weight > 0 else np.inf
    print(f"{model:<30} {weight:>10.4f}     {evidence_ratio:>8.1f}x")

print("\nNote: AIC weight represents the probability that the model is the best model.")
print("Evidence ratio shows how much better the best model is compared to others.")

## Visualize All Model Fits

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

colors = plt.cm.tab10(range(len(candidate_models)))

for i, model_name in enumerate(candidate_models):
    ax = axes[i]
    
    # Plot data
    ax.loglog(time, G_t_noisy, 'o', color='gray', 
              label='Data', markersize=5, alpha=0.5)
    
    # Plot model fit
    if model_name in comparison_pipeline.results:
        predictions = comparison_pipeline.results[model_name]['predictions']
        r2 = comparison_pipeline.results[model_name]['r_squared']
        aic = comparison_pipeline.results[model_name]['aic']
        
        ax.loglog(time, predictions, '-', color=colors[i], 
                 linewidth=2.5, label=f'Fit (R²={r2:.4f})')
        
        ax.set_xlabel('Time (s)', fontsize=10)
        ax.set_ylabel('G(t) (Pa)', fontsize=10)
        ax.set_title(f'{model_name}\nAIC = {aic:.1f}', fontsize=11)
        ax.legend(fontsize=9)
        ax.grid(True, alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[-1])

plt.suptitle('Model Comparison: All Fits', fontsize=14, y=1.00)
plt.tight_layout()
display(fig)
plt.close(fig)

## Residual Analysis

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 9))
axes = axes.flatten()

for i, model_name in enumerate(candidate_models):
    ax = axes[i]
    
    if model_name in comparison_pipeline.results:
        predictions = comparison_pipeline.results[model_name]['predictions']
        residuals = (G_t_noisy - predictions) / G_t_noisy * 100  # Percent error
        rmse = comparison_pipeline.results[model_name]['rmse']
        
        ax.semilogx(time, residuals, 'o-', color=colors[i], 
                   markersize=5, alpha=0.7, linewidth=1.5)
        ax.axhline(y=0, color='k', linestyle='--', linewidth=1)
        ax.axhline(y=5, color='r', linestyle=':', linewidth=1, alpha=0.5)
        ax.axhline(y=-5, color='r', linestyle=':', linewidth=1, alpha=0.5)
        
        ax.set_xlabel('Time (s)', fontsize=10)
        ax.set_ylabel('Relative Error (%)', fontsize=10)
        ax.set_title(f'{model_name}\nRMSE = {rmse:.2e} Pa', fontsize=11)
        ax.set_ylim([-15, 15])
        ax.grid(True, alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[-1])

plt.suptitle('Residual Analysis (red lines = ±5% error)', fontsize=14, y=1.00)
plt.tight_layout()
display(fig)
plt.close(fig)

## Parameter Comparison Across Models

Compare fitted parameters across models to understand physical interpretations.

In [ ]:
print("\n" + "="*80)
print("FITTED PARAMETERS FOR EACH MODEL")
print("="*80)

for model_name in candidate_models:
    if model_name in comparison_pipeline.results:
        print(f"\n{model_name.upper()}:")
        print("-" * 60)

        # Get model object to access Parameter objects with units
        model = comparison_pipeline.results[model_name]['model']

        # Iterate through ParameterSet
        for param in model.parameters.values():
            print(f"  {param.name:<15} = {param.value:>12.3e}  {param.units}")

## Complexity vs Performance Trade-off

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: R² vs number of parameters
ax1.plot(df_comparison_sorted['Parameters'], df_comparison_sorted['R²'], 
         'o-', markersize=10, linewidth=2)
for i, model in enumerate(df_comparison_sorted['Model']):
    ax1.annotate(model, 
                (df_comparison_sorted['Parameters'].iloc[i], 
                 df_comparison_sorted['R²'].iloc[i]),
                textcoords="offset points", xytext=(0,10), 
                ha='center', fontsize=8)
ax1.set_xlabel('Number of Parameters', fontsize=12)
ax1.set_ylabel('R² (Goodness of Fit)', fontsize=12)
ax1.set_title('Model Complexity vs Fit Quality', fontsize=13)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0.95, 1.001])

# Right: AIC comparison
x_pos = np.arange(len(df_comparison_sorted))
bars = ax2.bar(x_pos, df_comparison_sorted['AIC'], color=colors[:len(df_comparison_sorted)])
ax2.set_xlabel('Model', fontsize=12)
ax2.set_ylabel('AIC (lower is better)', fontsize=12)
ax2.set_title('Akaike Information Criterion', fontsize=13)
ax2.set_xticks(x_pos)
ax2.set_xticklabels(df_comparison_sorted['Model'], rotation=45, ha='right', fontsize=9)
ax2.grid(True, alpha=0.3, axis='y')

# Highlight best model
bars[0].set_edgecolor('red')
bars[0].set_linewidth(3)

plt.tight_layout()
display(fig)
plt.close(fig)


## Bonus: Oscillation Mode with Smart Initialization

Previously, fractional models (like FractionalZenerSolidSolid) failed to optimize in oscillation mode due to local minima in the non-convex optimization landscape created by Mittag-Leffler functions.

**Issue #9 Fix:** Smart initialization now extracts features from frequency-domain data (low/high frequency plateaus, transition frequency, slope) to provide data-driven initial parameter guesses. This guides the optimizer away from local minima.

Let's demonstrate that FZSS now works correctly in oscillation mode:

In [ ]:
# Generate frequency-domain oscillation data
omega = np.logspace(-2, 2, 50)  # 0.01 to 100 rad/s

# True parameters (same as relaxation example)
true_model_osc = FractionalZenerSolidSolid()
true_model_osc.set_params(Ge=Ge_true, Gm=Gm_true, alpha=alpha_true, tau_alpha=tau_alpha_true)

# Generate clean complex modulus G*(ω) = G' + iG"
G_star_clean = true_model_osc._predict_oscillation(
    omega, Ge_true, Gm_true, alpha_true, tau_alpha_true
)

# Add 5% noise to both G' and G"
np.random.seed(42)
G_star_np = np.array(G_star_clean)
noise_real = 0.05 * G_star_np[:, 0] * np.random.randn(len(omega))
noise_imag = 0.05 * G_star_np[:, 1] * np.random.randn(len(omega))
G_star_noisy = G_star_np.copy()
G_star_noisy[:, 0] += noise_real  # G' with noise
G_star_noisy[:, 1] += noise_imag  # G" with noise

print("Generated oscillation data (frequency domain):")
print(f"  Frequency range: {omega[0]:.2e} to {omega[-1]:.2e} rad/s")
print(f"  G' range: {G_star_noisy[:, 0].min():.2e} to {G_star_noisy[:, 0].max():.2e} Pa")
print(f"  G'' range: {G_star_noisy[:, 1].min():.2e} to {G_star_noisy[:, 1].max():.2e} Pa")
print()

# Fit FZSS in oscillation mode (smart initialization activates automatically)
print("Fitting FractionalZenerSolidSolid in oscillation mode...")
model_osc = FractionalZenerSolidSolid()
model_osc.fit(omega, G_star_noisy, test_mode='oscillation', max_iter=1000)

print("\n✓ Fit completed successfully!")
print("\nFitted parameters:")
Ge_fit = model_osc.parameters.get_value('Ge')
Gm_fit = model_osc.parameters.get_value('Gm')
alpha_fit = model_osc.parameters.get_value('alpha')
tau_alpha_fit = model_osc.parameters.get_value('tau_alpha')

print(f"  Ge        = {Ge_fit:.2e} Pa  (true: {Ge_true:.2e})")
print(f"  Gm        = {Gm_fit:.2e} Pa  (true: {Gm_true:.2e})")
print(f"  alpha     = {alpha_fit:.4f}    (true: {alpha_true:.4f})")
print(f"  tau_alpha = {tau_alpha_fit:.2e} s   (true: {tau_alpha_true:.2e})")

# Compute R²
G_pred = model_osc._predict_oscillation(omega, Ge_fit, Gm_fit, alpha_fit, tau_alpha_fit)
G_mag_data = np.sqrt(G_star_noisy[:, 0]**2 + G_star_noisy[:, 1]**2)
G_mag_pred = np.sqrt(np.array(G_pred[:, 0])**2 + np.array(G_pred[:, 1])**2)
ss_res = np.sum((G_mag_data - G_mag_pred)**2)
ss_tot = np.sum((G_mag_data - np.mean(G_mag_data))**2)
r2_osc = 1 - ss_res / ss_tot

print(f"\nR² = {r2_osc:.4f}")

if r2_osc > 0.95:
    print("\n🎉 SUCCESS: Oscillation mode optimization works correctly!")
    print("   Smart initialization successfully avoided local minima.")
else:
    print(f"\n⚠️  R² = {r2_osc:.4f} is below target (> 0.95)")

### Visualize Oscillation Mode Fit

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: Storage modulus G'
ax1.loglog(omega, G_star_noisy[:, 0], 'o', color='steelblue',
          label="Data (G')", markersize=6, alpha=0.6)
ax1.loglog(omega, np.array(G_pred[:, 0]), '-', color='red',
          label="Fit (G')", linewidth=2.5)
ax1.set_xlabel('Frequency ω (rad/s)', fontsize=12)
ax1.set_ylabel("Storage Modulus G' (Pa)", fontsize=12)
ax1.set_title(f"Storage Modulus (R² = {r2_osc:.4f})", fontsize=13)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Right: Loss modulus G"
ax2.loglog(omega, G_star_noisy[:, 1], 's', color='orange',
          label='Data (G")', markersize=6, alpha=0.6)
ax2.loglog(omega, np.array(G_pred[:, 1]), '-', color='darkred',
          label='Fit (G")', linewidth=2.5)
ax2.set_xlabel('Frequency ω (rad/s)', fontsize=12)
ax2.set_ylabel('Loss Modulus G" (Pa)', fontsize=12)
ax2.set_title('Loss Modulus', fontsize=13)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.suptitle('Oscillation Mode: FractionalZenerSolidSolid with Smart Initialization',
             fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

## Conclusion

In this notebook, we performed systematic model comparison for rheological data:

### Key Findings:

1. **Model Selection**: Information criteria (AIC, BIC) balance fit quality against complexity
2. **Best Model**: The model with lowest AIC/BIC is statistically preferred
3. **Complexity Trade-off**: More parameters improve R² but may overfit (penalized by AIC/BIC)
4. **Residual Patterns**: Systematic residuals indicate model inadequacy

### Interpretation Guidelines:

- **ΔA IC < 2**: Models are essentially equivalent
- **2 < ΔAIC < 4**: Moderate evidence favoring best model  
- **4 < ΔAIC < 7**: Strong evidence favoring best model
- **ΔAIC > 10**: Very strong evidence favoring best model

### When to Use Each Model:

- **Maxwell**: Purely viscous liquids with single relaxation time
- **Zener**: Materials with equilibrium modulus (gels, crosslinked polymers)
- **Fractional Models**: Materials with broad relaxation spectra (power-law behavior)
- **Complex Fractional**: Multi-modal relaxation with fractional components

### Recommendations:

1. Always compare multiple models systematically
2. Use information criteria (AIC/BIC) for objective selection
3. Examine residuals for systematic patterns
4. Consider physical interpretability of parameters
5. Validate selected model with independent test data

### Next Steps:

- Apply multi-technique fitting (see `multi_technique_fitting.ipynb`)
- Perform advanced sensitivity analysis (see `advanced_workflows.ipynb`)
- Explore Bayesian model selection with uncertainty quantification (Phase 3)